In [1]:
import os
import json
import sys
import requests
import re
from dotenv import load_dotenv
from anthropic import Anthropic
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import gradio as gr
import subprocess
import io

In [2]:
# Load environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "your_key_if_not_using_env")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY", "your_key_if_not_using_env")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN", "your_key_if_not_using_env")

In [3]:
openai = OpenAI()
claude = Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [4]:
def system_prompt_for(output_language, system):
    system_prompt = f"You are an assistant that reimplements Python code into high performance {output_language} code for an {system}. "
    system_prompt += f"Respond only with {output_language} code, use comments sparingly and do not provide any explanation other that occasional comments. "
    system_prompt += f"The {output_language} response needs to produce an identical output in the fastest possible time. Keep implementations of random numbers identical so that results match exactly"
    return system_prompt

In [5]:
print(system_prompt_for("C++", "M1 Mac"))

You are an assistant that reimplements Python code into high performance C++ code for an M1 Mac. Respond only with C++ code, use comments sparingly and do not provide any explanation other that occasional comments. The C++ response needs to produce an identical output in the fastest possible time. Keep implementations of random numbers identical so that results match exactly


In [6]:
def user_prompt_for(output_language, code):
    user_prompt = f"Rewrite this Python code into {output_language} with fastest possible implementation that produces identical output in the least time. "
    user_prompt += f"Respond only with {output_language} code, do not explain your code other than a few comments. "
    user_prompt += code
    return user_prompt


In [7]:
def generate_message(output_language, code, system):
    
    return [
        {"role": "system", "content": system_prompt_for(output_language, system)},
        {"role": "user", "content": user_prompt_for(output_language, code)}
    ]

In [8]:
def write_output(code, language):
    # Dictionary mapping languages to their file extensions
    extensions = {
        'c++': 'cpp',
        'python': 'py',
        'go': 'go',
    }
    
    # Normalize input language to lowercase for case-insensitive matching
    language = language.lower()
    
    # Get the appropriate extension
    if language not in extensions:
        raise ValueError(f"Unsupported language: {language}. Supported languages are: {', '.join(extensions.keys())}")
    
    extension = extensions[language]
    filename = f"optimized.{extension}"
    
    # Write the code to the appropriate file
    with open(filename, "w") as f:
        f.write(code)
    
    # Make the file executable if it's Python
    if language == 'python':
        import os
        os.chmod(filename, 0o755)
    
    return filename

In [48]:
code_snippet_1 = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [47]:
def stream_gpt(output_language, code, system="M1 Mac"):
    stream = openai.chat.completions.create(model=OPENAI_MODEL,
                                           messages=generate_message(output_language, code, system),
                                           stream=True)
    reply = ""
    first_line = True
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        if first_chunk and fragment.strip():  # Only process non-empty first chunk
            print("DEBUG - Before any cleaning:", repr(reply))
            
            # Step 1: Remove ```cpp or ```go
            reply = re.sub(r'^[\s\n]*```\s*(cpp|go)\s*\n*', '', reply, flags=re.IGNORECASE)
            print("DEBUG - After step 1:", repr(reply))
            
            # Step 2: Remove standalone cpp or go
            reply = re.sub(r'^[\s\n]*(cpp|go)\s*\n*', '', reply, flags=re.IGNORECASE)
            print("DEBUG - After step 2:", repr(reply))
            
            # Step 3: Remove any remaining ``` at start
            reply = re.sub(r'^[\s\n]*```\s*\n*', '', reply)
            print("DEBUG - After step 3:", repr(reply))
            
            # Step 4: Remove any remaining language identifier
            reply = re.sub(r'^[\s\n]*(cpp|go)\s*\n*', '', reply, flags=re.IGNORECASE)
            print("DEBUG - Final first chunk:", repr(reply))
            
            first_chunk = False
        
        # Always clean end markers
        reply = re.sub(r'```\s*$', '', reply)
        
        yield reply

In [54]:
result = stream_gpt("C++", code_snippet_1)
for chunk in result:
    print(chunk)
    


Before cleaning: ''
After first clean: ''
After second clean: ''
After final clean: ''


cpp
cpp

cpp
#include
cpp
#include <
cpp
#include <iostream
cpp
#include <iostream>

cpp
#include <iostream>
#include
cpp
#include <iostream>
#include <
cpp
#include <iostream>
#include <chrono
cpp
#include <iostream>
#include <chrono>


cpp
#include <iostream>
#include <chrono>

double
cpp
#include <iostream>
#include <chrono>

double calculate
cpp
#include <iostream>
#include <chrono>

double calculate(int
cpp
#include <iostream>
#include <chrono>

double calculate(int iterations
cpp
#include <iostream>
#include <chrono>

double calculate(int iterations,
cpp
#include <iostream>
#include <chrono>

double calculate(int iterations, int
cpp
#include <iostream>
#include <chrono>

double calculate(int iterations, int param
cpp
#include <iostream>
#include <chrono>

double calculate(int iterations, int param1
cpp
#include <iostream>
#include <chrono>

double calculate(int iterations, int param1,
cpp
#in

In [45]:
def stream_claude(output_language, code, system="M1 Mac"):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_prompt_for(output_language, system),
        messages=[{"role": "user", "content": user_prompt_for(output_language, code)}]
    )

    reply = ""
    first_line = True
    with result as stream:
        for text in stream.text_stream:
            reply += text
            if first_line:
                # Print initial state for debugging
                print("Before cleaning:", repr(reply[:50]))
                
                # Clean in steps with more aggressive patterns
                # First remove ```cpp or ```go with any spacing
                reply = re.sub(r'^[\s\n]*```\s*(cpp|go)\s*', '', reply, flags=re.IGNORECASE)
                print("After first clean:", repr(reply[:50]))
                
                # Then remove any standalone cpp or go
                reply = re.sub(r'^[\s\n]*(cpp|go)\s*', '', reply, flags=re.IGNORECASE)
                print("After second clean:", repr(reply[:50]))
                
                # Finally remove any remaining ``` markers at the start
                reply = re.sub(r'^[\s\n]*```\s*', '', reply)
                print("After final clean:", repr(reply[:50]))
                
                first_line = False
            reply = re.sub(r'```\s*$', '', reply)
            yield reply

In [37]:
exec(code_snippet_1)

Result: 3.141592658589
Execution Time: 13.271362 seconds


In [13]:
stream_gpt("Go", code_snippet_1)

<generator object stream_gpt at 0x14fdd4a60>

In [14]:
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [15]:
def execute_python(code):
    try:
        output = io.StringIO(code)
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()
        

In [16]:
import hashlib


def execute_golang(code):
    """
    Execute Go code and return its output.
    Uses a consistent temp directory for better performance with Go's build cache.
    """
    # Create a stable temp directory for Go builds if it doesn't exist
    temp_dir = os.path.join(os.getcwd(), 'temp_go_builds')
    os.makedirs(temp_dir, exist_ok=True)
    
    try:
        # Create a unique filename using a hash of the code
        file_hash = hashlib.md5(code.encode()).hexdigest()[:10]
        go_file = os.path.join(temp_dir, f"main_{file_hash}.go")
        executable = os.path.join(temp_dir, f"main_{file_hash}")
        
        # Write the code only if the file doesn't exist
        if not os.path.exists(go_file):
            with open(go_file, 'w') as f:
                f.write(code)
        
        # Compile and run in one step using 'go run' if it's a quick execution
        # or compile separately if the program will be run multiple times
        process = subprocess.run(
            ['go', 'run', go_file],
            capture_output=True,
            text=True,
            cwd=temp_dir
        )
        
        if process.returncode != 0:
            return f"Error:\n{process.stderr}"
            
        return process.stdout.strip()
        
    except Exception as e:
        return f"Execution error: {str(e)}"


In [17]:

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [18]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
.go {background-color: #305499;}
"""

In [19]:
def optimize(model, output_language, code):
    if model=="GPT":
        result = stream_gpt(output_language, code)
    elif model=="Claude":
        result = stream_claude(output_language, code)
    else:
        raise ValueError("Unknown Model")
    
    for stream_so_far in result:
        yield stream_so_far

In [55]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++ or Go")
    with gr.Row():
        python_code = gr.Textbox(label="Python Code:", value = python_hard, lines=10)
        cpp_or_go = gr.Textbox(label= "C++/Go Code:", lines = 10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select Model", value="GPT")
        output_language = gr.Dropdown(["C++", "Go"], label = "Select Language", value="C++")
        output_language.change(
        fn=lambda x: gr.Textbox(label=f"{x} Code:"), 
        inputs=[output_language], 
        outputs=[cpp_or_go]
        )
    with gr.Row():
        convert = gr.Button("Convert Code")
        
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
        go_run = gr.Button("Run Go")
    with gr.Row():
        python_out = gr.TextArea(label="Python Result", elem_classes=["Python"])
        cpp_out = gr.TextArea(label="C++ Result", elem_classes=["C++"])
        go_out = gr.TextArea(label="Go Result", elem_classes=["Go"])
    
    convert.click(optimize, inputs=[model, output_language, python_code], outputs=[cpp_or_go])
    python_run.click(execute_python, inputs = [python_code], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp_or_go], outputs=[cpp_out])
    go_run.click(execute_golang, inputs=[cpp_or_go], outputs=[go_out])

        


ui.launch(inbrowser=True)   



* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


Before cleaning: ''
After first clean: ''
After second clean: ''
After final clean: ''
